In [1]:
# This notebook have as a point to allow us scraping job sites, to give us ability to have a DB that references 
# all available computing jobs in France.
# We should add also a country classemnet of basic salries to compare ?? TBD
# Steps:
"""
Search on :
- indeed
- monster
with keys {'informatique', 'france'} 
- Find where is job ads 
- parse and get {title, salary, location, missions, company name}
"""
#!pip install pandas
#!pip install bs4 requests lxml 
!pip install mysql-connector-python


You should consider upgrading via the '/home/fbordjah/.virtualenvs/scraping_project/bin/python -m pip install --upgrade pip' command.


In [44]:
# Install libraries
#!pip install  lxml requests bs4  pandas selenium

import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import mysql.connector
import time
from selenium import webdriver
from selenium.webdriver.firefox.options import Options as FirefoxOptions
from selenium.webdriver.chrome.options import Options as ChromeOptions



In [45]:
# Utils:

#Compute the salary per month for INDEED
def compute_salary(card):
    salary_tmp = None
    final_salary = None
    if card.find({"span"},{"class":"salaryText"}) is not None:
        salary_tmp = card.find({"span"}, {"class":"salaryText"}).text.strip()
        salary_tmp_splited = salary_tmp.split(' ')
        result = salary_tmp_splited[0].replace('\xa0','')
        final_salary= int(result)
        if salary_tmp_splited[-1] != "an":
            final_salary = final_salary*12
    return final_salary

#Compute the salary per month for monster
def compute_salary_mon(salary_string):
    sal = None
    if salary_string is not None:
        splited = salary_string.split(' ')
        cleaned_string = splited[0].lstrip("€")
        sal = int(cleaned_string)
        if splited[-1].lower() != "an":
            sal = sal*12
    return sal


# configure Firefox Driver
## Pleach check your version chrom
def web_driver():
    # Add additional Options to the webdriver
    firefox_options = FirefoxOptions()
    # add the argument and make the browser Headless.
    firefox_options.add_argument("--headless")

    # Instantiate the Webdriver: Mention the executable path of the webdriver you have downloaded
    # if driver is in PATH, no need to provide executable_path
    driver = webdriver.Firefox(executable_path = "./geckodriver", options=firefox_options)
    return driver

# configure Chrome Webdriver
def configure_chrome_driver():
    # Add additional Options to the webdriver
    chrome_options = ChromeOptions()
    # add the argument and make the browser Headless.
    chrome_options.add_argument("--headless")
    # Instantiate the Webdriver: Mention the executable path of the webdriver you have downloaded
    # if driver is in PATH, no need to provide executable_path
    driver = webdriver.Chrome(executable_path="./chromedriver", options = chrome_options)
    return driver
# fetch soup request function
def do_request(url):
    try:
        res = requests.get(url)
        res.raise_for_status()
    except requests.exceptions.HTTPError as errh:
        print ("Http Error:",errh)
    except requests.exceptions.ConnectionError as errc:
        print ("Error Connecting:",errc)
    except requests.exceptions.Timeout as errt:
        print ("Timeout Error:",errt)
    except requests.exceptions.RequestException as err:
        print ("OOps: Something Else",err)
    # parse witrh soup
    return bs(res.content,"html")

# Simulate a scroll and click on cookies constement pop up
def fetch_salary(url):
    browser=web_driver()
    browser.get(url)
    browser.execute_script("window.scrollTo(0,document.body.scrollHeight)")
    try:
        browser.find_element_by_xpath("//button[text()='Fermer']").click()
    except: 
        print("No button")
        pass
    time.sleep(5)
    ad_detail = bs(browser.page_source, 'html')
    browser.close()
    salary = None
    try:
        salary = ad_detail.find({"div"},{"name":"value_salary"}).text
    except:
        print("No salary")
        pass
    return salary
    
indeed_url = "https://fr.indeed.com/emplois?q=informatique&l=France"
monster_url = "https://www.monster.fr/emploi/recherche/?q=Informatique"
apec_url = "https://www.apec.fr/candidat/recherche-emploi.html/emploi?niveauxExperience=101881&motsCles=informatique"

In [46]:
# search for jobs with key 'inforamtique' and 'france' location.
# INDEED
ads = []
for x in range(0,10,10):
    soup = do_request(indeed_url)
    cards = soup.find_all({"div"},{"class":"jobsearch-SerpJobCard unifiedRow row result"})
    for card in cards:
        ad = {}
        ad["ref"] = card.attrs['id'].lstrip('\n')
        ad["job"] =  card.h2.a.text.replace('\n','')
        ad["company"] = card.find({"div"},{"class":"sjcl"}).span.text.lstrip('\n')
        ad["location"] = card.find({"div"},{"class":"sjcl"}).find({"div"},{"class":"recJobLoc"}).attrs['data-rc-loc'].lstrip('\n')
        ad['origin'] = 'indeed'
        ad["link"] = 'https://fr.indeed.com'+card.find({"a"},{"class":"jobtitle turnstileLink"}).attrs['href'].lstrip('\n')
        ad["salary"] = compute_salary(card)
        ads.append(ad)


In [10]:
# Monster
# step 1: 
# fetch ads links in one page
soup_monster = do_request(monster_url)
allPostLink = soup_monster.findAll("h2", {'class', 'title'})
linksArray = []
for post in allPostLink:
    linksArray.append(post.a['href']) 

# step 2: 
# fetch data in one ad details
for linkUrl in linksArray:
    try:
        res = requests.get(linkUrl)
        res.raise_for_status()
    except:
        continue
    ad_detail = bs(res.text, 'html')
    ad = {}
    ad['ref'] = linkUrl.split('/')[-1]
    ad['job'] = ad_detail.find({"h1"},{"class":"job_title"}).text
    ad['company'] = ad_detail.find({"div"},{"name":"job_company_name"}).text
    ad['location'] = ad_detail.find({"div"},{"name":"job_company_location"}).text 
    ad['origin'] = 'monster'
    ad['link'] = linkUrl
    salary_string = fetch_salary("https://offre-demploi.monster.fr/technicien-r%C3%A9f%C3%A9rent-hydraulique-pneumatique-h-f-hazebrouck-nord-hauts-de-france-fr-page-personnel/222134414")
    ad['salary'] = compute_salary_mon(salary_string)
    ads.append(ad) 


https://offre-demploi.monster.fr/ingénieur-en-développement-doutils-de-systèmes-et-de-sécurité-informatique-oise-hauts-de-france-fr-armée-de-lair-et-de-lespace/221712124
30000


In [59]:
# Apec data
ads = []
baselink = "https://www.apec.fr"
browser=configure_chrome_driver()
browser.get(apec_url)
soup_apec = bs(browser.page_source, 'html')
cards = soup_apec.find_all({"a"},{"queryparamshandling":"merge"})
for card in cards:
    url = baselink + card.attrs['href']
    browser.get("https://www.apec.fr/candidat/recherche-emploi.html/emploi/detail-offre/165717969W?niveauxExperience=101881&motsCles=informatique&selectedIndex=17&page=0")
    detail_apec = bs(browser.page_source, 'html')
    ad = {}
    draft_ref = None
    try:
        draft_ref = detail_apec.find({"div"},{"class":"card-offer__text"}).div.text
    except:
        draft_ref =  detail_apec.find({"div"},{"class":"ref-offre"}).text
    ad['ref'] = draft_ref.split(' ')[4]
    ad['job'] = detail_apec.find({"h2"},{"class":"card-title mt-0"}).text
    lists = detail_apec.find({"ul"},{"class":"details-offer-list mb-20"}).findChildren()
    try:
        ad['company'] = lists[0].text
        ad['location'] = lists[3].text
    except:
        print("Erro in list")
    ad['origin'] = 'apec'
    ad['link'] = url
    # #### TODO: Parse to int (create a function to enhance it)
    ad['salary'] = detail_apec.find({"div"},{"class":"details-post"}).span.text
    ads.append(ad)
browser.close()

https://www.apec.fr/candidat/recherche-emploi.html/emploi/detail-offre/165698395W?niveauxExperience=101881&motsCles=informatique&selectedIndex=0&page=0
 Ref. Apec : 165717969W / Ref. Société : 1797831 
165717969W
https://www.apec.fr/candidat/recherche-emploi.html/emploi/detail-offre/165677438W?niveauxExperience=101881&motsCles=informatique&selectedIndex=1&page=0
 Ref. Apec : 165717969W / Ref. Société : 1797831 
165717969W
https://www.apec.fr/candidat/recherche-emploi.html/emploi/detail-offre/165747326W?niveauxExperience=101881&motsCles=informatique&selectedIndex=2&page=0
 Ref. Apec : 165717969W / Ref. Société : 1797831 
165717969W
https://www.apec.fr/candidat/recherche-emploi.html/emploi/detail-offre/165742010W?niveauxExperience=101881&motsCles=informatique&selectedIndex=3&page=0
 Ref. Apec : 165717969W / Ref. Société : 1797831 
165717969W
https://www.apec.fr/candidat/recherche-emploi.html/emploi/detail-offre/165733084W?niveauxExperience=101881&motsCles=informatique&selectedIndex=4&pag

In [51]:
df = pd.DataFrame(ads2)
df

,ref,job,company,location,origin,link,salary
0,165698395W,Développeur informatique F/H,Cabinet de recrutement MANPOWER Nantes Informa...,Nantes - 44,apec,https://www.apec.fr/candidat/recherche-emploi....,A négocier


,ref,job,company,location,origin,link,salary
0,pj_812912ff36556f1f,Ingénieur informatique H/F,IT SYSTEMES,Levallois-Perret (92),indeed,https://fr.indeed.com/pagead/clk?mo=r&ad=-6NYl...,40000.0
1,pj_cde5f8c7707419b8,Ingénieur Informatique,Promosoft Informatique,Orléans (45),indeed,https://fr.indeed.com/pagead/clk?mo=r&ad=-6NYl...,37000.0
2,pj_20cd3b9bd97ccabb,Technicien informatique Apple,Aldorande,Paris (75),indeed,https://fr.indeed.com/pagead/clk?mo=r&ad=-6NYl...,21000.0
3,pj_faaa832c710b15ef,Responsable Systèmes d' Offres (H/F),Engie Ineo,Achères (78),indeed,https://fr.indeed.com/pagead/clk?mo=r&ad=-6NYl...,NaN
4,pj_fee30ba12bfa50aa,Responsable des Systèmes d'Information H/F,Healthy Group,Levallois-Perret (92),indeed,https://fr.indeed.com/pagead/clk?mo=r&ad=-6NYl...,60000.0
5,pj_ca5fe3057b6a5e18,Technicien informatique H/F,MEDIPATH,Fréjus (83),indeed,https://fr.indeed.com/pagead/clk?mo=r&ad=-6NYl...,21600.0
6,p_4df776f7bfb63fd6,Technicien informatique H/F,Hemmersbach France,La Défense (92),indeed,https://fr.indeed.com/company/Hemmersbach-Fran...,22000.0
7,p_f0751e4d23bcee98,Administrateur / Administratrice système infor...,Pôle Emploi,Roissy-en-France (95),indeed,https://fr.indeed.com/rc/clk?jk=f0751e4d23bcee...,30000.0
8,p_b24b80b7fc677d70,Responsable informatique Dev & Admin,Axialys,Courbevoie (92),indeed,https://fr.indeed.com/company/AXIALYS/jobs/Res...,NaN
9,p_2da8b14a9ae9462b,Ingénieur / Ingénieure d'étude et développemen...,Pôle Emploi,Levallois-Perret (92),indeed,https://fr.indeed.com/rc/clk?jk=2da8b14a9ae946...,28000.0


In [ ]:
# Create database connection
#mydb = mysql.connector.connect(host="localhost",
                               port="3307",
                               user="scrap",
                               password="secret",
                               database="scrap")

# Test insertions
'''mycursor = mydb.cursor()
sql = "INSERT INTO ads (job, company) VALUES (%s, %s)"
val = ("John", "Highway 21")
mycursor.execute(sql, val)
mydb.commit()
'''